In [1]:
import pandas as pd
import pickle

In [ ]:
df = pd.read_csv("mixed_language_dataset.csv")

In [ ]:
df.drop_duplicates(inplace = True, ignore_index = True)

313

In [ ]:
train_df = df.sample(frac = 0.8)
train_df.nunique()

,0
text,250
language,2


In [ ]:
test_df = df.drop(train_df.index)
test_df.nunique()

,0
text,63
language,2


In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313471 sha256=22b8913d9b2cf0867560f53746cae65108d5e9bed18e3f8897ddbc2bde26dba6
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [3]:
import fasttext

In [ ]:
train_df = df.sample(frac = 0.8)

In [ ]:
with open("train_file_2.txt", "w") as file:
    for ind, row in train_df.iterrows():
        line = ""
        line += "__label__hindi " if row["language"] == "Hindi" else "__label__english "
        line += row["text"]
        file.write(line + "\n")

with open("test_file.txt", "w") as file:
    for ind, row in test_df.iterrows():
        line = ""
        line += "__label__hi  " if row["language"] == "Hindi" else "__label__en  "
        line += row["text"]
        file.write(line + "\n")

with open("train_df.pkl", "wb") as file:
    pickle.dump(train_df, file)

with open("test_df.pkl", "wb") as file:
    pickle.dump(test_df, file)


### Fine Tuning pretrained model

In [4]:
model = fasttext.train_supervised(input = "train_file.txt", epoch=25, lr=0.5, wordNgrams=2)

print(model.predict("Mai hungry feel kar raha hun."))
print(model.predict("Maine today morning mai milk aur bread ke sath Breakfast kiya"))

(('__label__hi',), array([0.97955251]))
(('__label__hi',), array([0.56985193]))


In [8]:
model.predict("Maine Dinner kar liya")

(('__label__hi',), array([0.66935128]))

In [9]:
model.predict("Mujhe Mineral Water ki jarurat hai")

(('__label__hi',), array([0.6983006]))

In [10]:
model.predict("Mujhe ice cream aur chocolate khani hai")

(('__label__hi',), array([0.61994505]))

In [11]:
with open("test_df.pkl", "rb") as file:
    test_df = pickle.load(file)

test_df.head()

,text,language
2,Mai kal dost ki cricket match attend kar raha ...,Hindi
4,"My favorite laptop is samosa, aur tumhara?",English
6,She loves message butter chicken at the beach.,English
7,"We watched a thriller movie last night, it was...",English
9,The pani puri was very yummy.,English


In [21]:
TP, TN, FP, FN = 0, 0, 0, 0
for _, row in test_df.iterrows():
    pred = model.predict(row["text"])[0][0]

    if pred == "__label__hi" and row["language"] == "Hindi":
        TP += 1
    elif pred == "__label__en" and row["language"] == "English":
        TN += 1
    elif pred == "__label__hi" and row["language"] == "English":
        FP += 1
    elif pred == "__label__en" and row["language"] == "Hindi":
        FN += 1

print("True Positives : ", TP)
print("True Negatives : ", TN)
print("False Positives : ", FP)
print("False Negatives : ", FN)
print("Precision: ", TP/(TP+FP))
print("Recall: ", TP/(TP+FN))
print("Accuracy: " , (TP+TN)/(TP+TN+FP+FN))
print("F1 Score: ", 2*TP/(2*TP+FP+FN))

True Positives :  18
True Negatives :  45
False Positives :  0
False Negatives :  0
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
F1 Score:  1.0
